In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys

In [36]:
from rmab.simulator import RMABSimulator, random_valid_transition, random_valid_transition_round_down, synthetic_transition_small_window
from rmab.uc_whittle import UCWhittle, UCWhittleFixed, UCWhittleMatch, NormPlusMatch
from rmab.ucw_value import UCWhittle_value
from rmab.baselines import optimal_policy, random_policy, WIQL, optimal_match_policy
from rmab.fr_dynamics import get_all_transitions
from rmab.utils import get_save_path, delete_duplicate_results


In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [5]:
if is_jupyter: 
    seed        = 42
    n_arms      = 8
    budget      = 3
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 30
    episode_len = 20
    n_epochs    = 10
    save_name = 'results_all_agents'
    match_prob = 0.5
    save_with_date = True 
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=8)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=20)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=30)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=10)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--save_name',      '-n', help='save name', type=str, default='results')
    parser.add_argument('--match_prob',      '-m', help='match probability', type=float, default=0.5)
    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    save_name   = args.save_name 
    save_with_date = args.use_date 
    match_prob = args.match_prob 



In [6]:
n_states = 2
n_actions = 2

In [7]:
all_population_size = 100 # number of random arms to generate
all_transitions = get_all_transitions(all_population_size)

In [8]:
all_transitions.shape

(100, 2, 2, 2)

In [9]:
all_features = np.arange(all_population_size)

In [10]:
np.random.seed(seed)
random.seed(seed)
simulator = RMABSimulator(all_population_size, all_features, all_transitions,
            n_arms, episode_len, n_epochs, n_episodes, budget, number_states=n_states, reward_style='match',match_probability=match_prob)

acting should always be good! 0.000 < 0.044
acting should always be good! 0.000 < 0.162
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [83 53 70 45 44 39 22 80]
cohort [45 80  5 29 78 35 86 14]
cohort [42 28 84 76 68 46 97 15]
cohort [51 29 14 74 15  3 31 87]
cohort [55  0 94  4 72 26 38  3]
cohort [69 66 55 63 30 61 42 29]
cohort [83 68 66 27 99 32 35 23]
cohort [31  1 98 11 36  4 84 58]
cohort [ 5 41 86 49 97 13 58 42]
cohort [22 71 78 83 30 48 34 29]


In [13]:
np.random.seed(seed)
random.seed(seed)
random_rewards = random_policy(simulator, n_episodes, n_epochs)
random_active_rate = simulator.total_active/(random_rewards.size * n_arms)

first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 1 1]
instance 0, ep 15, state [0 0 1 1 1 0 1 1]
instance 0, ep 16, state [1 1 1 1 0 1 1 0]
instance 0, ep 17, state [0 0 0 1 0 0 0 0]
instance 0, ep 18, state [0 1 1 0 1 1 1 0]
instance 0, ep 19, state [1 1 0 1 0 0 1 1]
instance 0, ep 20, state [1 0 0 1 1 0 1 0]
instance 0, ep 21, state [0 1 1 1 0 1 1 1]
instance 0, ep 22, state [0 0 0 0 0 0 0 1]
instance 0, ep 23, state [1 0 0 1

In [15]:
np.random.seed(seed)
random.seed(seed)
optimal_match_reward = optimal_match_policy(simulator, n_episodes, n_epochs, discount)
optimal_match_active_rate = simulator.total_active/(optimal_match_reward.size*n_arms)

first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 1 1]
instance 0, ep 15, state [0 0 1 1 1 0 1 1]
instance 0, ep 16, state [1 1 1 1 0 1 1 0]
instance 0, ep 17, state [0 0 0 1 0 0 0 0]
instance 0, ep 18, state [0 1 1 0 1 1 1 0]
instance 0, ep 19, state [1 1 0 1 0 0 1 1]
instance 0, ep 20, state [1 0 0 1 1 0 1 0]
instance 0, ep 21, state [0 1 1 1 0 1 1 1]
instance 0, ep 22, state [0 0 0 0 0 0 0 1]
instance 0, ep 23, state [1 0 0 1

In [25]:
np.random.seed(seed)
random.seed(seed)
optimal_reward = optimal_policy(simulator, n_episodes, n_epochs, discount)
optimal_active_rate = simulator.total_active/(optimal_reward.size*n_arms)

first state [0 1 1 1 1 1 1 1]
   state [0 1 1 1 1 1 1 1] state_WI [  0.39   0.47   0.47   0.26   0.45 -10.   -10.   -10.  ] sorted [2 1 4]
   state [0 1 1 0 1 0 1 1] state_WI [  0.39   0.47   0.47   0.71   0.45   0.52 -10.   -10.  ] sorted [3 5 2]
   state [0 1 1 1 1 1 0 0] state_WI [  0.39   0.47   0.47   0.26   0.45 -10.     0.49   0.82] sorted [7 6 2]
   state [0 1 1 1 0 0 0 0] state_WI [0.39 0.47 0.47 0.26 0.45 0.52 0.49 0.82] sorted [7 5 6]
   state [0 1 1 0 0 1 0 1] state_WI [  0.39   0.47   0.47   0.71   0.45 -10.     0.49 -10.  ] sorted [3 6 2]
   state [1 0 1 1 0 0 1 0] state_WI [  0.26   0.54   0.47   0.26   0.45   0.52 -10.     0.82] sorted [7 1 5]
   state [0 1 0 0 0 1 1 0] state_WI [  0.39   0.47   0.58   0.71   0.45 -10.   -10.     0.82] sorted [7 3 2]
   state [1 0 0 1 0 1 0 1] state_WI [  0.26   0.54   0.58   0.26   0.45 -10.     0.49 -10.  ] sorted [2 1 6]
   state [0 1 1 1 0 0 1 1] state_WI [  0.39   0.47   0.47   0.26   0.45   0.52 -10.   -10.  ] sorted [5 2 1]
   st

In [16]:
np.random.seed(seed)
random.seed(seed)
wiql_rewards = WIQL(simulator, n_episodes, n_epochs)
wiql_active_rate = simulator.total_active/(wiql_rewards.size*n_arms)

first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 1 1]
instance 0, ep 15, state [0 0 1 1 1 0 1 1]
instance 0, ep 16, state [1 1 1 1 0 1 1 0]
instance 0, ep 17, state [0 0 0 1 0 0 0 0]
instance 0, ep 18, state [0 1 1 0 1 1 1 0]
instance 0, ep 19, state [1 1 0 1 0 0 1 1]
instance 0, ep 20, state [1 0 0 1 1 0 1 0]
instance 0, ep 21, state [0 1 1 1 0 1 1 1]
instance 0, ep 22, state [0 0 0 0 0 0 0 1]
instance 0, ep 23, state [1 0 0 1

In [17]:
np.random.seed(seed)
random.seed(seed)
ucw_extreme_rewards = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='extreme')
ucw_extreme_active_rate = simulator.total_active/(ucw_extreme_rewards.size*n_arms)

solving UCWhittle using method: extreme
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 0 0 1 1]  | s'  [0 1 0 0 0 0 1 0]  | r  0.5    | WI  [0.898 0.898 0.898 0.9   0.898 0.898 0.898 0.9  ]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 0 1 0 1 1 0]  | s'  [0 0 0 1 0 1 0 1]  | r  0.75    | WI  [  0.898   0.898   0.898   0.9     0.898   0.898   0.898 -10.   ]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0,

In [18]:
np.random.seed(seed)
random.seed(seed)
ucw_ucb_rewards = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB')
ucw_ucb_active_rate = simulator.total_active/(ucw_ucb_rewards.size*n_arms)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 0 0 1 1 1]  | s'  [0 1 0 0 0 1 1 0]  | r  0.75    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 1 0 0 0 0 1 1]  | s'  [0 1 0 1 0 1 0 1]  | r  0.75    | WI  [-0.     0.044 -0.    -0.    -0.    -0.    -0.    -0.   ]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 

In [19]:
np.random.seed(seed)
random.seed(seed)
rewards_without_norm = UCWhittleFixed(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',norm_confidence=False)
ucw_without_norm_active_rate = simulator.total_active/(rewards_without_norm.size*n_arms)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 0 0 1 1 1]  | s'  [0 1 0 0 0 1 1 0]  | r  0.75    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 1 0 1 0 0 1]  | s'  [1 0 0 0 1 0 0 1]  | r  0.75    | WI  [-0.    -0.     0.001 -0.     0.005 -0.    -0.    -0.   ]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 

In [20]:
np.random.seed(seed)
random.seed(seed)
rewards_with_norm = UCWhittleFixed(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',norm_confidence=True)
ucw_with_norm_active_rate = simulator.total_active/(rewards_with_norm.size*n_arms)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 1 0 0 1]  | s'  [0 0 0 1 1 0 1 0]  | r  0.75    | WI  [-10.      0.362   0.194   0.843   0.877 -10.    -10.      1.   ]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 1 0 1 1 0 0 0]  | s'  [1 0 1 0 0 0 0 1]  | r  0.0    | WI  [  0.316   0.59    0.349   0.799   0.797 -10.    -10.    -10.   ]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]

In [21]:
np.random.seed(seed)
random.seed(seed)
rewards_match_heuristic = UCWhittleMatch(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB')
match_heuristic_active_rate = simulator.total_active/(rewards_match_heuristic.size*n_arms)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 1 0 0 1]  | s'  [0 1 0 1 1 0 0 1]  | r  0.875
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 0 1 1 0 0 1]  | s'  [0 0 0 1 0 0 0 1]  | r  0.75
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 0]
instance 0, ep 14, state [0 1 1 0 1 1 1 1]
instance 0, ep 15, state [0 0 1 1 1 0 1 1]
---------------------------------------------------
0 300  | 

In [59]:
np.random.seed(seed)
random.seed(seed)
rewards_combined = NormPlusMatch(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',lamb=0.25)
combined_active_rate = simulator.total_active/(rewards_combined.size*n_arms)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 0 1 1 0]  | s'  [0 0 0 0 0 1 1 1]  | r  0.75    | WI  [-10.    -10.     -0.      0.709  -0.265   1.      0.762   0.116]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 0 1 0 1 1 0]  | s'  [1 0 0 1 0 1 1 0]  | r  0.875    | WI  [-10.    -10.    -10.      0.791  -0.341   1.      0.623 -10.   ]
instance 0, ep 11, state [1 0 1 1 0 1 1 1]
instance 0, ep 12, state [0 0 1 1 0 0 1 0]
instance 0, ep 13, state [1 1 1 1 0 1 1 

In [60]:
mean_rewards = {'random_rewards': np.mean(random_rewards), 
 'optimal_rewards': np.mean(optimal_reward), 
 'optimal_match_rewards': np.mean(optimal_match_reward),
 'wiql_rewards': np.mean(wiql_rewards), 
 'extreme_rewards': np.mean(ucw_extreme_rewards), 
 'ucb_rewards': np.mean(ucw_ucb_rewards), 
 'fixed_rewards': np.mean(rewards_without_norm), 
 'norm_rewards': np.mean(rewards_with_norm), 
 'predicted_optimal_match_rewards': np.mean(rewards_match_heuristic), 
 'combined_rewards': np.mean(rewards_combined)}

In [61]:
active_rates = {'random_rewards': random_active_rate, 
 'optimal_rewards': optimal_active_rate, 
 'optimal_match_rewards': optimal_match_active_rate,
 'wiql_rewards': wiql_active_rate, 
 'extreme_rewards': ucw_extreme_active_rate, 
 'ucb_rewards': ucw_ucb_active_rate, 
 'fixed_rewards': ucw_without_norm_active_rate, 
 'norm_rewards': ucw_with_norm_active_rate, 
 'predicted_optimal_match_rewards': match_heuristic_active_rate, 
 'combined_rewards': combined_active_rate}

In [62]:
mean_rewards 

{'random_rewards': 0.6488144758735441,
 'optimal_rewards': 0.6788477537437604,
 'optimal_match_rewards': 0.7653078202995008,
 'wiql_rewards': 0.610108153078203,
 'extreme_rewards': 0.7121256239600665,
 'ucb_rewards': 0.5919301164725458,
 'fixed_rewards': 0.5913685524126456,
 'norm_rewards': 0.6601705490848586,
 'predicted_optimal_match_rewards': 0.7400790349417637,
 'combined_rewards': 0.6859608985024959}

In [47]:
std_rewards = {'random_rewards': np.std(random_rewards), 
 'optimal_rewards': np.std(optimal_reward), 
 'optimal_match_reward': np.mean(optimal_match_reward), 
 'wiql_rewards': np.std(wiql_rewards), 
 'extreme_rewards': np.std(ucw_extreme_rewards), 
 'ucb_rewards': np.std(ucw_ucb_rewards), 
 'fixed_rewards': np.std(rewards_without_norm), 
 'norm_rewards': np.std(rewards_with_norm), 
 'predicted_optimal_match_rewards': np.std(rewards_match_heuristic)}

In [20]:
data = {
    'mean_reward': mean_rewards, 
    'std_reward': std_rewards,
    'active_rate': active_rates, 
    'parameters': 
        {'seed'      : seed,
        'n_arms'    : n_arms,
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'match_prob': match_prob} 
}

In [23]:
save_path = get_save_path('matching',save_name,seed,use_date=save_with_date)

In [25]:
delete_duplicate_results('matching',save_name,data)

In [26]:
json.dump(data,open('../results/'+save_path,'w'))